## Exercise 5 ~ Nibblet Oracle

In [1]:
import socket
from random import choice

In [2]:
Sciper = "247565"
ct = "42a208e4c6beb1d7f3bd718f523345b73dec3f422ab9f0e1ec1b1278b02eb0d8e71296458e4733ed0444a5f8dc2df167abf4ad17fefdf3a1f62e180f8a3ee41c"

Let's get the server methods :

In [3]:
def connect_server(server_name, port, message):
    server = (server_name, int(port)) #calling int is required when using Sage
    s = socket.create_connection(server)
    s.send(message + "\n")
    response=""
    while True: #data might come in several packets, need to wait for all of it
        data = s.recv(1024)
        if len(data) == 0:
            break
        if data[-1] == '\n': 
            response += data[:-1]  
            break
        response += data
    s.close()
    return response


def encryption_query(sciper, pt):
    server_name = "lasecpc25.epfl.ch"
    port = "5559"
    message = sciper + " " + pt
    response = connect_server(server_name, port, message)
    #print(response)
    return response

In [4]:
print "Number of blocks :", len(ct) / 16

Number of blocks : 8


The ciphertext has 8 blocks.

Here is a helper method to get random hexadecimal string :

In [5]:
def rand_hex(l=16):
    hex_chars = "0123456789abcdef"
    h = []
    for i in range(l):
        h.append(choice(hex_chars))
    return ''.join(h)

And helper methods to convert hexadecimal to binary, and the other way around.

In [6]:
def hex_to_binary(h):
    final_length = len(h) * 4
    b = bin(int(h, 16))[2:]
    while len(b) < final_length:
        b = '0' + b
    return b

def binary_to_hex(b):
    final_length = len(b) / 4
    h = hex(int(''.join(b), 2))[2:]
    if h[-1] == 'L':
        h = h[:-1]
    while len(h) < final_length:
        h = '0' + h
    return h

Let's try to see what impact flipping each bit has on the ciphertext :

In [7]:
# This is how many flips we do for each bit 
ITER_NUM = 4

all_flipped_bits = []

for bit_to_flip in range(64): # For each bit in [0..64]
    flipped_bits = dict()
    for i in range(ITER_NUM): # Do this ITER_NUM times
        # Take a random hexadecimal value and encrypt it
        first_plain = rand_hex()
        first_enc = encryption_query(Sciper, first_plain)
        # Now flip the bit we are analysing, and encrypt that
        second_plain_bin_list = list(hex_to_binary(first_plain))
        if second_plain_bin_list[bit_to_flip] == '0':
            second_plain_bin_list[bit_to_flip] = '1'
        else:
            second_plain_bin_list[bit_to_flip] = '0'
        second_plain = binary_to_hex(second_plain_bin_list)
        second_enc = encryption_query(Sciper, second_plain)
        # Log the differences
        first_enc_bin = hex_to_binary(first_enc)
        second_enc_bin = hex_to_binary(second_enc)
        for j in range(len(first_enc_bin)):
            if first_enc_bin[j] != second_enc_bin[j]:
                if j in flipped_bits:
                    flipped_bits[j] += 1
                else:
                    flipped_bits[j] = 1
            else:
                if j not in flipped_bits:
                    flipped_bits[j] = 0
    all_flipped_bits.append([flipped_bits[x] for x in range(64)])

These represent the data we have aggregated in the previous cell.

Each line represents one bit that we flipped.

A "1" means the bit might be affected, and a "0" that it was never affected.

In [8]:
all_flipped_bits_str = []
for f in range(len(all_flipped_bits)):
    s = ''
    for i in range(len(all_flipped_bits[f])):
        if all_flipped_bits[f][i] == 0:
            s += '0'
        else:
            s += '1'
    all_flipped_bits_str.append(s)
    print s

0100000011000001011100001000000001100000000000010000100101000011
0000100100100110000010000010010010000100000001100001000000010100
1001011000000000000001100001001000000011100000001000000000101000
1001011000000000000001100001001100000011101000001000000000101000
0010000000011000000000010100100000011000010100000100011010000000
0000100100100110000010000010010010000000000001100001000000010100
0000100100100110000010000010010000000100000001100001000000010100
0100000011000001001100001000000000100000000000010000100101000011
1001011000000000000001100001001100000011101000001000000000101000
0010000000011000100000010100100000011000010110000100010010000000
1001011000000000000001100001001100000011101000001000000000100000
0000100100100110000010000010010010000100000001100011000000010100
1001011000000000000001000001001100000011101000001000000000101000
0000100100100110000000000010010010000100000001100011000000010100
1001011000000000000001100001001100000011101000001000000000101000
0000100100100110000010000

There seem to be many similar lines.

One reason could be that a subset of input bits only affect a certain other subset of output bits. Let's test this intuition :

In [9]:
groups = []
for i in range(64):
    for j in range(len(all_flipped_bits_str)):
        for k in range(len(all_flipped_bits_str)):
            if all_flipped_bits_str[j][i] == '1':
                if all_flipped_bits_str[k][i] == '1':
                    found_group = False
                    for (l, m) in groups:
                        if (k in l) and (j not in l):
                            l.add(j)
                        if (j in l):
                            m.add(i)
                            found_group = True
                            break
                    if found_group == False:
                        groups.append((set([j]), set([i])))
groups = [(list(sorted(s)), list(sorted(m))) for (s, m) in groups]
print groups

[([2, 3, 8, 10, 12, 14, 19, 20, 21, 22, 25, 36, 53, 58, 59, 61], [0, 3, 5, 6, 21, 22, 27, 30, 31, 38, 39, 40, 42, 48, 58, 60]), ([0, 7, 16, 18, 23, 26, 27, 31, 32, 39, 40, 42, 45, 47, 48, 62], [1, 8, 9, 15, 17, 18, 19, 24, 33, 34, 47, 52, 55, 57, 62, 63]), ([4, 9, 17, 24, 28, 29, 33, 37, 41, 43, 44, 50, 54, 55, 56, 60], [2, 11, 12, 16, 23, 25, 28, 35, 36, 41, 43, 44, 49, 53, 54, 56]), ([1, 5, 6, 11, 13, 15, 30, 34, 35, 38, 46, 49, 51, 52, 57, 63], [4, 7, 10, 13, 14, 20, 26, 29, 32, 37, 45, 46, 50, 51, 59, 61])]


Gotcha ! There are four different groups of bits that affect four different groups of bits ! This will give us a way to reverse the ciphertext.

We can use a "simple" exhaustive search : we have reduced our search space to $2^{16}$ values.

Let's first create all possible values of the 16 bits :

In [10]:
plain_m = ''

for i in range(2 ** 16):
    bin_i = '{0:016b}'.format(i)
    temp_m = [' ' for _ in range(64)]
    for (g, _) in groups:
        for j in range(len(g)):
            temp_m[g[j]] = bin_i[j]
    plain_m += ''.join(temp_m)

plain_hex_m = binary_to_hex(plain_m)

Now we can encrypt all of those values, and we should be able to deconstruct our ciphertext using the output.

In [11]:
enc_m = ''
split = 16
for i in range(split):
    m = plain_hex_m[i * len(plain_hex_m) / split:(i + 1) * len(plain_hex_m) / split]
    enc_m += encryption_query(Sciper, m)

Now we reconstruct the plaintext :

In [12]:
my_flipped_bits = []
final_bin = [' ' for _ in range(len(ct) * 4)]
for block in range(len(ct) / 16): # For each 64-bit block of ct
    b = ct[block * 16:(block + 1) * 16]
    block_bin = hex_to_binary(b)
    temp_bin = [' ' for _ in range(len(block_bin))]
    for i in range(len(enc_m) / 16): # For each 64-bit block of enc_m
        enc_block = enc_m[i * 16:(i+1) * 16]
        enc_block_bin = hex_to_binary(enc_block)
        for g in groups:
            is_a_match = True
            for group_loc in g[1]:
                if enc_block_bin[group_loc] != block_bin[group_loc]:
                    is_a_match = False
            if is_a_match:
                h = plain_hex_m[i * 16:(i+1) * 16]
                h_bin = hex_to_binary(h)
                for group_loc in g[0]:
                    temp_bin[group_loc] = h_bin[group_loc]
    final_bin[block * len(block_bin):(block + 1) * len(block_bin)] = temp_bin

In [14]:
print binary_to_hex(final_bin)
print encryption_query(Sciper, binary_to_hex(final_bin)) == ct

e985260398b0d8f10d3762bc63308194791b3cf02a8c87bcdb3f8accf14f552a249076f97abd4324c3d9f134c2925c66105b17803931716a832b51f1288b9716
True
